In [1]:
import config
import event
import handler
import strategy
import portfolio
import risk
import analysis as an
from datetime import datetime

#SQL & API credentials. Store in config file.

access_token = config.access_token
account_id = config.account_id

sql_host = config.sql_host
sql_user = config.sql_user_analysis
sql_password = config.sql_password_analysis

In [ ]:
for i in range(0, df.shape[0]):
    if df.iloc[i]['State'] != df.iloc[i-1]['State']:
        three_hour_trend = an.state(an.selectlast('H3', df['time'].iloc[i], 10))
        if df.iloc[i]['State'] == three_hour_trend:
            print(df['time'].iloc[i], df.iloc[i]['State'], an.state(an.selectlast('H3', df['time'].iloc[i], 10))) 

In [2]:
dh = handler.DataHandler()
queue = event.event_queue()
strategy = strategy.Strategy()

In [87]:
queue.add_to_queue(dh.get_latest_rate())

In [88]:
while len(queue.queue) != 0:
    next_event = queue.next_event()
    
    if next_event.type == 'tick':
        #Call strategy
        queue.add_to_queue(strategy.generate_signal(next_event))
        print('Tick Event Popped')

    elif next_event.type == 'signal':
        #Call risk
        print('Signal Event Popped')

    elif next_event.type == 'trade':
        #Call portfolio
        print('Trade Event Popped')

    elif next_event.type == 'order':
        #Call execution
        print('Order Event Popped')

    elif next_event.type == 'fill':
        #Wait in execution function until response received
        #Call portfolio
        print('Fill Event Popped')

    else:
        print('Queue Error')


Tick Event Popped
Signal Event Popped


In [89]:
len(queue.queue)

0

In [93]:
next_event.type

'signal'